# LlamaAPI tutorial

reference: https://docs.llama-api.com/quickstart



### Settings

In [36]:
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta


# Load environment variables from .env file
dotenv_path = os.path.join(os.path.dirname(__name__), '../../', '.env')  # Assuming .env is located one level up
load_dotenv(dotenv_path)

# Now you can access your environment variables
llama_api_token = os.environ.get('LLAMA_API_TOKEN')


### API request

In [75]:
import json
from llamaapi import LlamaAPI

# Initialize the SDK
llama = LlamaAPI(llama_api_token)

#### Example 1

In [ ]:
# Build the API request
api_request_json = {
    "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"},
    ],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "days": {
                        "type": "number",
                        "description": "for how many days ahead you wants the forecast",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
            },
            "required": ["location", "days"],
        }
    ],
    "stream": False,
    "function_call": "get_current_weather",
}

# Execute the Request
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))

In [42]:
answer = response.json()['choices'][0]['message']
print(json.dumps(answer, indent=2))

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_flight_info",
    "arguments": {
      "loc_origin": "Amsterdam",
      "loc_destination": "New York"
    }
  }
}


#### Example 2

In [32]:
# Build the Request
api_request_json = {
  'model': 'llama-13b-chat',
  'functions': [
      {
          "name": "get_flight_info",
          "description": "Get flight information between two locations",
          "parameters": {
              "type": "object",
              "properties": {
                  "loc_origin": {
                      "type": "string",
                      "description": "The departure airport, e.g. DUS"
                  },
                  "loc_destination": {
                      "type": "string",
                      "description": "The destination airport, e.g. HAM"
                  }
              },
          "required": ["loc_origin", "loc_destination"]
          }
      }
  ],
  'function_call': {'name': 'get_flight_info'},
  'messages': [
      {'role': 'user', 'content': "When's the next flight from Amsterdam to New York?"}],
}

# Execute the Request
response = llama.run(api_request_json)
# Save the answer
output = response.json()['choices'][0]['message']

In [37]:
def get_flight_info(loc_origin, loc_destination):
  flight_info = {
  "loc_origin": loc_origin,
  "loc_destination": loc_destination,
  "datetime": str(datetime.now() + timedelta(hours=2)),
  "airline": "KLM",
  "flight_number": "KL 123",
  }

  return json.dumps(flight_info)

arguments = output['function_call']['arguments']
origin = arguments.get('loc_origin', None)
destination = arguments.get('loc_destination', None)

## Call the method
flight_info = get_flight_info(origin, destination)


In [39]:
users_promt = "When's the next flight from Amsterdam to New York?"
# function_description = 

second_api_request_json = {
  "model": "llama-70b-chat",
  "messages": [
    {"role": "user", "content": users_promt},
    {"role": "function", "name": output['function_call']['name'], "content": flight_info}
  ],
  # "functions": function_description,
}

second_request = llama.run(second_api_request_json)
print(second_request.json()['choices'][0]['message']['content'])


The next flight from Amsterdam to New York is KLM flight KL 123, departing Amsterdam at 06:00 AM on March 29, 2024, and arriving at New York John F. Kennedy International Airport at 08:30 AM. Would you like me to book a flight for you?


#### Custom

In [76]:
api_request_json = {
    "messages": [
        {"role": "user", "content": "list 4 cooking recipes"},
        {"role": "assistant", "content": "Chicken Parmesan, Lemon Garlic Shrimp, Beef Stroganoff, Lasagna"},
        {"role": "user", "content": "list 30 cooking recipes"},

    ],
}


response = llama.run(api_request_json)
# print(json.dumps(response.json(), indent=2))

In [78]:
recipe_list = response.json()['choices'][0]['message']['content']
print(recipe_list[:50])

1. Chicken Parmesan
2. Lemon Garlic Shrimp
3. Beef


In [82]:
api_request_json = {
    "messages": [
        {"role": "user", "content": "list cooking recipes"},
        {"role": "assistant", "content": recipe_list},
        {"role": "user", "content": "give me description, preperation time, cooking time, difficulty level, ingredient(name, quantity, unit), tags(e.g. tasty, italian, healthy) for each recipe"},
    ],
}

response = llama.run(api_request_json)

In [87]:
answers = []
answer = response.json()['choices'][0]['message']['content']
answers.append(answer)

messages = [
    {"role": "user", "content": "list cooking recipes"},
    {"role": "assistant", "content": recipe_list},
    {"role": "user", "content": "give me description, preperation time, cooking time, difficulty level, ingredient(name, quantity, unit), tags(e.g. tasty, italian, healthy) for each recipe"},
    ]

new_message = {"role": "assistant", "content": answer}
messages.append(new_message)


In [95]:

api_request_json = {
    "messages": messages,
}

response = llama.run(api_request_json)

In [97]:

answer = response.json()['choices'][0]['message']['content']
print(answer)

<Response [200]>


In [98]:
response.json()

{'created': 1711654318,
 'model': 'llama-13b-chat',
 'usage': {'prompt_tokens': 11, 'completion_tokens': 748, 'total_tokens': 759},
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '1. Chicken Parmesan\n\t* Description: Breaded and fried chicken topped with melted mozzarella cheese and marinara sauce, served with pasta.\n\t* Preperation Time: 20 minutes\n\t* Cooking Time: 25 minutes\n\t* Difficulty Level: Easy\n\t* Ingredients:\n\t\t+ Chicken breasts (4), boneless and skinless (1 lb)\n\t\t+ All-purpose flour (1 cup)\n\t\t+ Breadcrumbs (1 cup)\n\t\t+ Eggs (2)\n\t\t+ Salt (to taste)\n\t\t+ Pepper (to taste)\n\t\t+ Olive oil (for frying)\n\t\t+ Marinara sauce (1 cup)\n\t\t+ Mozzarella cheese (1 cup, shredded)\n\t\t+ Pasta (1 lb)\n\t* Tags: tasty, italian, easy\n2. Lemon Garlic Shrimp\n\t* Description: Shrimp sautéed in garlic butter and lemon, served with rice.\n\t* Preperation Time: 15 minutes\n\t* Cooking Time: 10 minutes\n\t* Difficulty Level: Easy\n\t* Ingr

In [100]:
api_request_json = {
    "messages": [
        {"role": "user", "content": "list cooking recipes"},
        {"role": "assistant", "content": recipe_list},
        {"role": "user", "content": "give me description, preperation time, cooking time, difficulty level, ingredient(name, quantity, unit), tags(e.g. tasty, italian, healthy) for each recipe"},
        {"role": "assistant", "content": answers[0]},
    ],
}



response = llama.run(api_request_json)

In [101]:

answer = response.json()['choices'][0]['message']['content']
print(answer)

25 minutes.

Here is the list of ingredients, preparation time, cooking time, difficulty level, and tags for each recipe:

1. Chicken Parmesan:
	* Ingredients: chicken breasts, all-purpose flour, breadcrumbs, eggs, salt, pepper, olive oil, marinara sauce, mozzarella cheese, pasta.
	* Preparation time: 20 minutes
	* Cooking time: 25 minutes
	* Difficulty level: Easy
	* Tags: tasty, italian, easy
2. Lemon Garlic Shrimp:
	* Ingredients: shrimp, garlic, butter, lemon juice, salt, pepper, rice.
	* Preparation time: 15 minutes
	* Cooking time: 10 minutes
	* Difficulty level: Easy
	* Tags: tasty, seafood, easy
3. Beef Stroganoff:
	* Ingredients: beef strips, mushrooms, onions, garlic, butter, cream, egg noodles, salt, pepper.
	* Preparation time: 25 minutes
	* Cooking time: 20 minutes
	* Difficulty level: Medium
	* Tags: tasty, beef, medium
4. Lasagna:
	* Ingredients: pasta, meat sauce, ricotta cheese, mozzarella cheese, parmesan cheese, egg, salt, pepper.
	* Preparation time: 30 minutes
	* C